In [252]:
import pandas as pd
from itertools import compress

class Transaction:
    def __init__(self, shopid, userid):
        self.shopid = shopid
        self.userid = userid
    def __repr__(self):
        return f"Transaction({self.shopid}, {self.userid})"

In [256]:
d_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
order_df = pd.read_csv('data\order_brush_order.csv', parse_dates=['event_time'], date_parser=d_parser)

c:\users\boonkiat\pycharmprojects\pyshop\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [257]:
trans_class_arr = []
for i in range(len(order_df)):
    trans_class_arr.append(Transaction(order_df['shopid'][i], order_df['userid'][i]))
order_df['Trans(shopid, userid)'] = trans_class_arr
order_df

,orderid,shopid,userid,event_time,"Trans(shopid, userid)"
0,31076582227611,93950878,30530270,2019-12-27 00:23:03,"Transaction(93950878, 30530270)"
1,31118059853484,156423439,46057927,2019-12-27 11:54:20,"Transaction(156423439, 46057927)"
2,31123355095755,173699291,67341739,2019-12-27 13:22:35,"Transaction(173699291, 67341739)"
3,31122059872723,63674025,149380322,2019-12-27 13:01:00,"Transaction(63674025, 149380322)"
4,31117075665123,127249066,149493217,2019-12-27 11:37:55,"Transaction(127249066, 149493217)"
...,...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59,"Transaction(110868129, 193218997)"
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20,"Transaction(91639906, 3541807)"
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52,"Transaction(29391773, 135908070)"
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31,"Transaction(61556313, 27306783)"


In [258]:
df = pd.concat([order_df['event_time'], order_df['Trans(shopid, userid)']], sort=False, axis='columns')
df['Trans(shopid, userid)'] = df['Trans(shopid, userid)'].astype(str)
df_bytrans = df.groupby('Trans(shopid, userid)')
df.set_index('event_time', inplace=True)
df

,"Trans(shopid, userid)"
event_time,
2019-12-27 00:23:03,"Transaction(93950878, 30530270)"
2019-12-27 11:54:20,"Transaction(156423439, 46057927)"
2019-12-27 13:22:35,"Transaction(173699291, 67341739)"
2019-12-27 13:01:00,"Transaction(63674025, 149380322)"
2019-12-27 11:37:55,"Transaction(127249066, 149493217)"
...,...
2019-12-28 23:17:59,"Transaction(110868129, 193218997)"
2019-12-28 19:07:20,"Transaction(91639906, 3541807)"
2019-12-28 08:17:52,"Transaction(29391773, 135908070)"


In [276]:
bool_list = list(df['Trans(shopid, userid)'].value_counts() != 1)
index_list = list(df['Trans(shopid, userid)'].value_counts().index)
filt_list = list(compress(index_list, filt_list))
filt = (df['Trans(shopid, userid)'].isin(filt_list))
suspected_df = df.loc[filt]
suspected_df

,"Trans(shopid, userid)"
event_time,
2019-12-27 11:37:55,"Transaction(127249066, 149493217)"
2019-12-27 22:10:04,"Transaction(286003, 42990566)"
2019-12-27 13:37:26,"Transaction(213154942, 119257587)"
2019-12-27 18:07:09,"Transaction(39938958, 67554544)"
2019-12-27 22:33:08,"Transaction(193010376, 56699817)"
...,...
2019-12-28 11:58:20,"Transaction(147941492, 1837980)"
2019-12-28 09:09:40,"Transaction(143281052, 203644830)"
2019-12-28 12:27:05,"Transaction(61556313, 9616959)"


In [283]:
suspected_df = df.resample('180min').agg({'Trans(shopid, userid)':'value_counts'})
suspected_df.rename(columns={'Trans(shopid, userid)':'Count'}, inplace=True)
suspected_df.columns

Index(['Count'], dtype='object')

In [284]:
suspected_df['Count'].nlargest(20)

event_time           Trans(shopid, userid)            
2019-12-31 09:00:00  Transaction(136564914, 178491887)    17
2019-12-30 15:00:00  Transaction(16001939, 205729485)     10
2019-12-27 18:00:00  Transaction(68609828, 183440131)      8
2019-12-27 21:00:00  Transaction(192785588, 94497233)      8
2019-12-31 12:00:00  Transaction(64394533, 194833170)      8
2019-12-27 21:00:00  Transaction(137762841, 62080153)      7
2019-12-28 12:00:00  Transaction(145777302, 201343856)     7
                     Transaction(147941492, 213787697)     7
2019-12-30 00:00:00  Transaction(147941492, 208806468)     7
2019-12-30 15:00:00  Transaction(68609828, 183440131)      7
2019-12-31 15:00:00  Transaction(16001939, 205729485)      7
2019-12-27 00:00:00  Transaction(147941492, 212970579)     6
2019-12-27 09:00:00  Transaction(147941492, 2812894)       6
2019-12-27 12:00:00  Transaction(54615708, 2973390)        6
2019-12-27 18:00:00  Transaction(147941492, 88732127)      6
2019-12-27 21:00:00  Transacti